In [1]:
#!pip install tensorflow==2.14
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
#py파일이어야 임포트 됨
from KNN_functions import *
from Customized_CNN import *

In [3]:
def AMTCount(data, isLearn):
    if(not isLearn):
        #print("data\n\n", data)
        #print("df_train_global\n\n", df_train_global)
#         df_train_temp = df_train_global.copy()
#         del df_train_temp["DAY"]
#         print("df_train_temp\n\n", df_train_temp)
        data = pd.concat([df_train_global, data], axis=0)
    amtCounted = pd.DataFrame([], columns=["CC_NUM", "AMT_COUNT"])
    if(isLearn):
        amtCounted["AMT_COUNT"] = ((data["CC_NUM"].to_frame())).value_counts()
        #amtCounted["AMT_COUNT"] = (data.to_frame()).value_counts()
        amtCounted["CC_NUM"] = (data["CC_NUM"].value_counts()).index
        amtCounted["CC_NUM"] = amtCounted["CC_NUM"].astype("int")
        amtCounted.reset_index(drop=True, inplace = True)
        amtCounted = amtCounted.sort_values(by=["CC_NUM"] ,ascending=True)
        #print("@@@@@@@@@@@@@@@@@", amtCounted)
        return amtCounted
    else: #data는 1행임, amtCounted는 빈 df
        amtCounted["AMT_COUNT"] = (data["CC_NUM"]).value_counts()
        #amtCounted["AMT_COUNT"] = (data.to_frame()).value_counts()
        amtCounted["CC_NUM"] = (data["CC_NUM"].value_counts()).index
        amtCounted["CC_NUM"] = amtCounted["CC_NUM"].astype("int64")
        amtCounted.reset_index(drop=True, inplace = True)
        amtCounted = amtCounted.sort_values(by=["CC_NUM"] ,ascending=True)
        return amtCounted

In [4]:
def FraudModel(data, isLearn, alpha, beta, gamma, delta1, delta2, omega, epsilon, mu):
    
    if(not isLearn):
        #data datatype is Series
        data = data.to_frame().T
    amtCount = AMTCount(data, isLearn)
    
    
    if(isLearn):
        KNNforFraud(data, isLearn, beta, delta1, epsilon)
        DifferenceforFraud(data, isLearn, gamma, delta2, omega, mu)
        return 2
    else:
        currentCCNUM = data["CC_NUM"]
        currentCCNUM = currentCCNUM.item()
        amtCount.reset_index(drop=True, inplace=True)
        #print("@#$$%#@$^#%$^#$^\n", amtCount)
        amt_count_temp = np.nan
        for row in amtCount.itertuples():
            #print(row.Index, row.CC_NUM, row.AMT_COUNT)
            if(currentCCNUM == row.CC_NUM):
                amt_count_temp = row.AMT_COUNT
        
        if(amtCount[amtCount["CC_NUM"] == currentCCNUM].empty or amt_count_temp < alpha):# 이게 다 Empty뜸
            #print("Decision value is___", amtCount[amtCount["CC_NUM"] == currentCCNUM])
            fraud_predict = KNNforFraud(data, isLearn, beta, delta1, epsilon)
        else:
            #print("Decision value is___", amtCount[amtCount["CC_NUM"] == currentCCNUM])
            fraud_predict = DifferenceforFraud(data, isLearn, gamma, delta2, omega, mu)
        return fraud_predict

In [5]:
global df_train_x_global
global df_train_y_global
global df_val_x_global
global df_val_y_global


df_train_x_global = pd.read_csv("../data/splitted/Fraud_Detection_train_features.csv")
df_train_y_global = pd.read_csv("../data/splitted/Fraud_Detection_train_target.csv")
df_val_x_global = pd.read_csv("../data/splitted/Fraud_Detection_validation_features.csv")
df_val_y_global = pd.read_csv("../data/splitted/Fraud_Detection_validation_target.csv")

#다루기 쉽게 하기 위해 IS_FRAUD를 int형으로 변경
df_train_y_global = df_train_y_global.astype("int")
df_val_y_global = df_val_y_global.astype("int")

global df_train_global
global df_val_global

df_train_global = pd.concat([df_train_x_global, df_train_y_global], axis=1)
df_val_global = pd.concat([df_val_x_global, df_val_y_global], axis=1)

model_result = FraudModel(data=df_train_global, isLearn=True, alpha=4, beta=7, gamma=4, delta1=0.5, delta2=0.5, omega=6, epsilon=5, mu=3)

Epoch 1/5
876/876 [==============================] - 1s 568us/step - loss: 0.2231 - accuracy: 0.9644
Epoch 2/5
876/876 [==============================] - 0s 560us/step - loss: 0.0376 - accuracy: 0.9850
Epoch 3/5
876/876 [==============================] - 0s 563us/step - loss: 0.0337 - accuracy: 0.9857
Epoch 4/5
876/876 [==============================] - 0s 560us/step - loss: 0.0330 - accuracy: 0.9861
Epoch 5/5
876/876 [==============================] - 0s 557us/step - loss: 0.0327 - accuracy: 0.9866
DifferenceforFraud START
ColumnsExtractor_CNN START
parameter is : 
        TRANS_YEAR  IS_COVID_YEAR  TRANS_MONTH  TRANS_DAY  \
0            2019          False            1         10   
1            2019          False            1         29   
2            2020           True           10          7   
3            2019          False            9          1   
4            2019          False            6          5   
...           ...            ...          ...        ...   
28009 

conv_row_idx :  100
conv_row_idx :  200
conv_row_idx :  300
conv_row_idx :  400
conv_row_idx :  500
conv_row_idx :  600
conv_row_idx :  700
conv_row_idx :  800
conv_row_idx :  900
Multiplier START
differenceSequence_fraud :     FRAUD_TODAY
0           1.0
0           0.0
0           0.0
0           0.0
0           0.0
..          ...
0           1.0
0           0.0
0           0.0
0           1.0
0           0.0

[16922 rows x 1 columns]
differenceSequence_differenceSequence_
            DIFF     DIFF2
0      0.214108  0.214108
1      0.665559  0.665559
2      0.861143  0.861143
3      0.457400  0.457400
4      0.234811  0.234811
...         ...       ...
16917  0.237786  0.237786
16918  0.240601  0.240601
16919  0.225895  0.225895
16920  0.166780  0.166780
16921  0.183779  0.183779

[16922 rows x 2 columns]


## 이제 validation을 해 보자
### FraudModel당 하나의 row(데이터)를 넣어야 한다!

In [6]:
predict_list = []
# for row_idx in range(df_val_x_global.shape[0]):
for row_idx in range(30):
    model_predicted=FraudModel(data=df_val_global.loc[row_idx], isLearn=False, alpha=4, beta=7, gamma=4, delta1=0.5, delta2=0.5, omega=6, epsilon=5, mu=3)
    predict_list.append(model_predicted)

print(predict_list)
print(type(predict_list))
    

DifferenceforFraud START
ColumnsExtractor_CNN START
parameter is : 
   TRANS_YEAR IS_COVID_YEAR TRANS_MONTH TRANS_DAY TRANS_DAY_SIMPLIFIED  \
0       2019         False           4        29                    2   

  TRANS_HOUR TRANS_HOUR_SIMPLIFIED CATEGORY     AMT GENDER  CITY CITY_POP  \
0          1                     0        1  344.48   True  Mesa      129   

  STATE           JOB AGE         CC_NUM   log_AMT BC_CITY_POP IS_FRAUD  
0     2  Cartographer  57  4079773899158  5.842036    3.767202        1  
reduced_data   TRANS_YEAR TRANS_MONTH TRANS_DAY         CC_NUM  DAY     AMT  IS_FRAUD
0       2019           4        29  4079773899158  118  344.48         1
reduced_datareduced_data

   TRANS_YEAR TRANS_MONTH TRANS_DAY         CC_NUM  DAY     AMT  IS_FRAUD
0       2019           4        29  4079773899158  118  344.48         1
day_:  118
i 0
ccnum_conserved_data.loc[i, DAY] :  36
<class 'int'>
day_:  118
i 1
ccnum_conserved_data.loc[i, DAY] :  42
<class 'int'>
day_:  118
i 

reduced_datareduced_data

   TRANS_YEAR TRANS_MONTH TRANS_DAY          CC_NUM  DAY    AMT  IS_FRAUD
0       2019           8        31  36721445180895  242  50.89         0
day_:  242
i 0
ccnum_conserved_data.loc[i, DAY] :  10
<class 'int'>
day_:  242
i 1
ccnum_conserved_data.loc[i, DAY] :  63
<class 'int'>
day_:  242
i 2
ccnum_conserved_data.loc[i, DAY] :  66
<class 'int'>
day_:  242
i 3
ccnum_conserved_data.loc[i, DAY] :  72
<class 'int'>
day_:  242
i 4
ccnum_conserved_data.loc[i, DAY] :  78
<class 'int'>
day_:  242
i 5
ccnum_conserved_data.loc[i, DAY] :  97
<class 'int'>
day_:  242
i 6
ccnum_conserved_data.loc[i, DAY] :  97
<class 'int'>
day_:  242
i 7
ccnum_conserved_data.loc[i, DAY] :  97
<class 'int'>
day_:  242
i 8
ccnum_conserved_data.loc[i, DAY] :  97
<class 'int'>
day_:  242
i 9
ccnum_conserved_data.loc[i, DAY] :  97
<class 'int'>
day_:  242
i 10
ccnum_conserved_data.loc[i, DAY] :  97
<class 'int'>
day_:  242
i 11
ccnum_conserved_data.loc[i, DAY] :  97
<class 'int'>
day_:  24

reduced_datareduced_data

   TRANS_YEAR TRANS_MONTH TRANS_DAY           CC_NUM  DAY    AMT  IS_FRAUD
0       2019           6         9  180046165512893  159  97.35         0
day_:  159
i 0
ccnum_conserved_data.loc[i, DAY] :  61
<class 'int'>
day_:  159
i 1
ccnum_conserved_data.loc[i, DAY] :  255
<class 'int'>
DifferenceGenerator START
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^data_converted :      AMT:
0  12.76
1  12.76
2  12.76
3  97.35
@@@@@@@@@@@@@@@@@@diff_scalar :  253.76999999999998
<class 'numpy.float64'>
@@@@@@@@@@@@@@@@@@fraud_of_diff_scalar :  0
<class 'numpy.int32'>
HHHHHHHHHHHHHHHHHHHHHHHHHH
           DIFF  FRAUD_TODAY
16917  1014.58          1.0
16918  1026.56          0.0
16919   963.98          0.0
16920   712.41          1.0
16921   784.75          0.0
0       253.77          0.0
differenceSequence_scaled_temp :             DIFF     DIFF2
0      0.035685  0.035685
1      0.110926  0.110926
2      0.143524  0.143524
3      0.076233  0.076233
4      0.039135  0.039135
...         ..

reduced_datareduced_data

   TRANS_YEAR TRANS_MONTH TRANS_DAY           CC_NUM  DAY    AMT  IS_FRAUD
0       2019          10         1  213124978348176  273  68.88         0
day_:  273
i 0
ccnum_conserved_data.loc[i, DAY] :  27
<class 'int'>
day_:  273
i 1
ccnum_conserved_data.loc[i, DAY] :  58
<class 'int'>
day_:  273
i 2
ccnum_conserved_data.loc[i, DAY] :  116
<class 'int'>
day_:  273
i 3
ccnum_conserved_data.loc[i, DAY] :  122
<class 'int'>
day_:  273
i 4
ccnum_conserved_data.loc[i, DAY] :  142
<class 'int'>
day_:  273
i 5
ccnum_conserved_data.loc[i, DAY] :  147
<class 'int'>
day_:  273
i 6
ccnum_conserved_data.loc[i, DAY] :  148
<class 'int'>
day_:  273
i 7
ccnum_conserved_data.loc[i, DAY] :  148
<class 'int'>
day_:  273
i 8
ccnum_conserved_data.loc[i, DAY] :  148
<class 'int'>
day_:  273
i 9
ccnum_conserved_data.loc[i, DAY] :  148
<class 'int'>
day_:  273
i 10
ccnum_conserved_data.loc[i, DAY] :  173
<class 'int'>
day_:  273
i 11
ccnum_conserved_data.loc[i, DAY] :  181
<class 'int

reduced_datareduced_data

   TRANS_YEAR TRANS_MONTH TRANS_DAY           CC_NUM  DAY   AMT  IS_FRAUD
0       2020           7        29  345933964507467  575  8.33         0
day_:  575
i 0
ccnum_conserved_data.loc[i, DAY] :  11
<class 'int'>
day_:  575
i 1
ccnum_conserved_data.loc[i, DAY] :  15
<class 'int'>
day_:  575
i 2
ccnum_conserved_data.loc[i, DAY] :  48
<class 'int'>
day_:  575
i 3
ccnum_conserved_data.loc[i, DAY] :  77
<class 'int'>
day_:  575
i 4
ccnum_conserved_data.loc[i, DAY] :  89
<class 'int'>
day_:  575
i 5
ccnum_conserved_data.loc[i, DAY] :  155
<class 'int'>
day_:  575
i 6
ccnum_conserved_data.loc[i, DAY] :  159
<class 'int'>
day_:  575
i 7
ccnum_conserved_data.loc[i, DAY] :  160
<class 'int'>
day_:  575
i 8
ccnum_conserved_data.loc[i, DAY] :  160
<class 'int'>
day_:  575
i 9
ccnum_conserved_data.loc[i, DAY] :  160
<class 'int'>
day_:  575
i 10
ccnum_conserved_data.loc[i, DAY] :  160
<class 'int'>
day_:  575
i 11
ccnum_conserved_data.loc[i, DAY] :  160
<class 'int'>
da

reduced_datareduced_data

   TRANS_YEAR TRANS_MONTH TRANS_DAY        CC_NUM  DAY   AMT  IS_FRAUD
0       2019           5         5  567868110212  124  5.22         0
day_:  124
i 0
ccnum_conserved_data.loc[i, DAY] :  6
<class 'int'>
day_:  124
i 1
ccnum_conserved_data.loc[i, DAY] :  22
<class 'int'>
day_:  124
i 2
ccnum_conserved_data.loc[i, DAY] :  55
<class 'int'>
day_:  124
i 3
ccnum_conserved_data.loc[i, DAY] :  60
<class 'int'>
day_:  124
i 4
ccnum_conserved_data.loc[i, DAY] :  82
<class 'int'>
day_:  124
i 5
ccnum_conserved_data.loc[i, DAY] :  114
<class 'int'>
day_:  124
i 6
ccnum_conserved_data.loc[i, DAY] :  118
<class 'int'>
day_:  124
i 7
ccnum_conserved_data.loc[i, DAY] :  132
<class 'int'>
DifferenceGenerator START
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^data_converted :       AMT:
0    6.04
1   14.72
2  181.82
3    5.22
@@@@@@@@@@@@@@@@@@diff_scalar :  705.5799999999999
<class 'numpy.float64'>
@@@@@@@@@@@@@@@@@@fraud_of_diff_scalar :  0
<class 'numpy.int32'>
HHHHHHHHHHHHHHHHHHHHHHH

reduced_datareduced_data

   TRANS_YEAR TRANS_MONTH TRANS_DAY            CC_NUM  DAY   AMT  IS_FRAUD
0       2020           3        10  3558881852140592  434  5.08         0
day_:  434
i 0
ccnum_conserved_data.loc[i, DAY] :  138
<class 'int'>
day_:  434
i 1
ccnum_conserved_data.loc[i, DAY] :  155
<class 'int'>
day_:  434
i 2
ccnum_conserved_data.loc[i, DAY] :  329
<class 'int'>
day_:  434
i 3
ccnum_conserved_data.loc[i, DAY] :  329
<class 'int'>
day_:  434
i 4
ccnum_conserved_data.loc[i, DAY] :  329
<class 'int'>
day_:  434
i 5
ccnum_conserved_data.loc[i, DAY] :  330
<class 'int'>
day_:  434
i 6
ccnum_conserved_data.loc[i, DAY] :  330
<class 'int'>
day_:  434
i 7
ccnum_conserved_data.loc[i, DAY] :  330
<class 'int'>
day_:  434
i 8
ccnum_conserved_data.loc[i, DAY] :  330
<class 'int'>
day_:  434
i 9
ccnum_conserved_data.loc[i, DAY] :  330
<class 'int'>
day_:  434
i 10
ccnum_conserved_data.loc[i, DAY] :  330
<class 'int'>
day_:  434
i 11
ccnum_conserved_data.loc[i, DAY] :  478
<class 'i

reduced_datareduced_data

   TRANS_YEAR TRANS_MONTH TRANS_DAY       CC_NUM  DAY    AMT  IS_FRAUD
0       2019           7        10  60495593109  190  43.01         0
day_:  190
i 0
ccnum_conserved_data.loc[i, DAY] :  69
<class 'int'>
day_:  190
i 1
ccnum_conserved_data.loc[i, DAY] :  127
<class 'int'>
day_:  190
i 2
ccnum_conserved_data.loc[i, DAY] :  174
<class 'int'>
day_:  190
i 3
ccnum_conserved_data.loc[i, DAY] :  189
<class 'int'>
day_:  190
i 4
ccnum_conserved_data.loc[i, DAY] :  367
<class 'int'>
DifferenceGenerator START
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^data_converted :      AMT:
0  17.41
1  77.24
2  43.02
3  43.01
@@@@@@@@@@@@@@@@@@diff_scalar :  94.08
<class 'numpy.float64'>
@@@@@@@@@@@@@@@@@@fraud_of_diff_scalar :  0
<class 'numpy.int32'>
HHHHHHHHHHHHHHHHHHHHHHHHHH
           DIFF  FRAUD_TODAY
16917  1014.58          1.0
16918  1026.56          0.0
16919   963.98          0.0
16920   712.41          1.0
16921   784.75          0.0
0        94.08          0.0
differenceSequence

reduced_datareduced_data

   TRANS_YEAR TRANS_MONTH TRANS_DAY         CC_NUM  DAY   AMT  IS_FRAUD
0       2020           4         2  4562827002127  457  9.07         1
day_:  457
i 0
ccnum_conserved_data.loc[i, DAY] :  17
<class 'int'>
day_:  457
i 1
ccnum_conserved_data.loc[i, DAY] :  80
<class 'int'>
day_:  457
i 2
ccnum_conserved_data.loc[i, DAY] :  102
<class 'int'>
day_:  457
i 3
ccnum_conserved_data.loc[i, DAY] :  131
<class 'int'>
day_:  457
i 4
ccnum_conserved_data.loc[i, DAY] :  456
<class 'int'>
day_:  457
i 5
ccnum_conserved_data.loc[i, DAY] :  457
<class 'int'>
DifferenceGenerator START
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^data_converted :       AMT:
0   28.74
1   12.58
2  714.82
3    9.07
@@@@@@@@@@@@@@@@@@diff_scalar :  2835.65
<class 'numpy.float64'>
@@@@@@@@@@@@@@@@@@fraud_of_diff_scalar :  1
<class 'numpy.int32'>
HHHHHHHHHHHHHHHHHHHHHHHHHH
           DIFF  FRAUD_TODAY
16917  1014.58          1.0
16918  1026.56          0.0
16919   963.98          0.0
16920   712.41          1